# Voici un notebook de démarrage pour vous aider à débuter le hackathon.

**[Inspiré par l'article de blog d'AI Geek (wishesh)](https://levelup.gitconnected.com/building-a-private-ai-chatbot-2c071f6715ad)**

In [3]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
# Installer les packages requis
%%capture
!pip install -q datasets transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain sentence-transformers gradio
!pip install -U langchain-community -q
!pip install pdfplumber -q
!pip install PyMuPDF -q
!pip install pypdf -q

In [ ]:
!pip install -U langchain-huggingface -q

In [ ]:
# importer les libraries requises
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader
from langchain.llms import HuggingFacePipeline
from huggingface_hub import notebook_login
from langchain import HuggingFacePipeline
from langchain.vectorstores import Chroma
from datasets import load_dataset
import gradio as gr
import chromadb
import locale
import torch
import os
import json

In [ ]:
# Pour corriger une erreur Unicode dans Google Colab
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Assurez-vous de créer un compte Hugging Face et de générer votre jeton pour vous connecter ici.
notebook_login()

# Charger le modèle

In [ ]:
# Spécifiez le nom du modèle Huggingface
model_name = "meta-llama/Llama-2-7b-chat-hf"
#model_name = "mistralai/Mistral-7B-Instruct-v0.1"

###### other models:
# "Trelis/Llama-2-7b-chat-hf-sharded-bf16"
# "bn22/Mistral-7B-Instruct-v0.1-sharded"
# "mistralai/Mistral-7B-Instruct-v0.2"
# "HuggingFaceH4/zephyr-7b-beta"
# "anakin87/zephyr-7b-alpha-sharded"
# function for loading 4-bit quantized model

def load_quantized_model(model_name: str):
    """
    :param model_name: Nom ou chemin du modèle à charger.
    :return: Modèle quantifié chargé.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        # Charger en 4 bits
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

# Fonction pour initialiser le tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialise le tokenizer avec le nom de modèle spécifié.

    :param model_name: Nom ou chemin du modèle pour l'initialisation du tokenizer.
    :return: Tokenizer initialisé.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Définir l'id du token de début de phrase
    return tokenizer


In [ ]:
# Charger le modèle
model = load_quantized_model(model_name)

# Initialiser le tokenizer
tokenizer = initialize_tokenizer(model_name)

# Spécifier les ids des tokens de fin
stop_token_ids = [0]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Créer des embeddings

###### Charger les données

In [ ]:
# En supposant des données localement
# Décompresser le dossier des données
!unzip -q /content/Data.zip

In [ ]:
DATA_PATH = "/content/Data"
JSONS_PATH = f"{DATA_PATH}/data_jsons"
PDFS_PATH = f"{DATA_PATH}/data_pdfs"

In [ ]:
pdfs = [doc for doc in os.listdir(PDFS_PATH) if doc.endswith('.pdf')]
print(f"Il y a {len(pdfs)} fichiers pdf dans le {PDFS_PATH}.")

Il y a 11 fichiers pdf dans le /content/Data/data_pdfs.


In [ ]:
jsons = [doc for doc in os.listdir(JSONS_PATH) if doc.endswith('.json')]
print(f"Il y a {len(jsons)} fichiers json dans le répertoire {JSONS_PATH}.")

Il y a 1184 fichiers json dans le répertoire /content/Data/data_jsons.


In [ ]:
# Code montrant comment extraire du texte d'un fichier pdf en utilisant pdfplumber.
import pdfplumber

example_pdf = f"{PDFS_PATH}/{pdfs[0]}"
print(f"Traitement en cours : {example_pdf}")

pdf_text = ""
try:
    with pdfplumber.open(example_pdf) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:  # Si du texte est extrait, l'ajouter à pdf_text
                pdf_text += text + "\n"
except Exception as e:
    print(f"Erreur lors de l'ouverture du fichier PDF : {e}")

# Affichage des 1000 premiers caractères du texte extrait
print(pdf_text[:1000])


Traitement en cours : /content/Data/data_pdfs/AU_mediation.pdf
www.droit‐afrique.com OHADA
OHADA
Acte uniforme relatif à la médiation
Acte fait à Conakry le 23 novembre 2017
[NB ‐ Acte uniforme relatif à la médiation, signé à Conakry le 23 novembre 2017]
Chapitre 1 ‐ Définitions et champ d’application
Art.1.‐ Définitions
Au sens du présent Acte uniforme :
 a) le terme « médiation » désigne tout processus, quelle que soit son appellation,
dans lequel les parties demandent à un tiers de les aider à parvenir à un règlement
amiable d’un litige, d’un rapport conflictuel ou d’un désaccord (ci‐après
le « différend ») découlant d’un rapport juridique, contractuel ou autre ou lié à un tel
rapport, impliquant des personnes physiques ou morales, y compris des entités
publiques ou des Etats ;
 b) le terme « médiateur » désigne tout tiers sollicité pour mener une médiation
quelle que soit l’appellation ou la profession de ce tiers dans l’État Partie concerné.
La médiation peut être mise en œuvr

In [ ]:
import fitz  # PyMuPDF

# Charger le fichier PDF
doc = fitz.open(f"{PDFS_PATH}/{pdfs[0]}")

# Extraire les métadonnées
metadata = doc.metadata
print("📄 Métadonnées du PDF :")
for key, value in metadata.items():
    print(f"{key}: {value}")

# Extraire le texte complet de toutes les pages
full_text = ""
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    full_text += f"\n--- Page {page_num + 1} ---\n{text}"

# Imprimer ou sauvegarder le texte extrait
print("\n📝 Texte extrait :")
print(full_text)

# Optionnel : Fermer le document
doc.close()

📄 Métadonnées du PDF :
format: PDF 1.5
title: OHADA - Acte uniforme relatif a la mediation (www.droit-afrique.com)
author: Droit Afrique
subject: 
keywords: 
creator: PScript5.dll Version 5.2.2
producer: Acrobat Distiller 10.1.1 (Windows)
creationDate: D:20171130144100+01'00'
modDate: D:20171130144200+01'00'
trapped: 
encryption: None

📝 Texte extrait :

--- Page 1 ---
 
www.droit‐afrique.com 
OHADA
  
 
Acte uniforme relatif à la médiation 
1
OHADA 
Acte uniforme relatif à la médiation 
Acte fait à Conakry le 23 novembre 2017 
 
[NB ‐ Acte uniforme relatif à la médiation, signé à Conakry le 23 novembre 2017] 
Chapitre 1 ‐ Définitions et champ d’application 
Art.1.‐ Définitions 
Au sens du présent Acte uniforme : 
a) le terme « médiation » désigne tout processus, quelle que soit son appellation, 
dans lequel les parties demandent à un tiers de les aider à parvenir à un règlement 
amiable d’un litige, d’un rapport conflictuel ou d’un désaccord (ci‐après 
le « différend ») découlant d’

In [ ]:
import json

# Exemple de code montrant comment extraire le contenu d'un fichier json.
def extract_json_content(json_path):
    # Lire et traiter le fichier json
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    return json_data

# Exemple de fichier json à traiter
example_json = f"{JSONS_PATH}/{jsons[0]}"
print(f"Traitement en cours : {example_json}")

# Extraction du contenu du fichier json
json_content = extract_json_content(example_json)

# Afficher le contenu extrait du json
print(json_content)

Traitement en cours : /content/Data/data_jsons/OHADA-COURCOMMUNEDEJUSTICEETDARBITRAGE-20210225-0272021.json
{'title': "| OHADA, Cour commune de justice et d'arbitrage, 25 février 2021, 027/2021", 'content': 'ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICEET D’ARBITRAGE(CCJA)Deuxième chambreAudience publique du 25 février 2021Pourvois : n°219/2019/PC du 14/08/2019n°220/2019/PC du 14/08/2019Affaire : ONYX Développement(Conseil : Maître Agi LAWEL CHEKOU KORE, Avocat à la Cour)contreAc X BAb Ad CSociété de Transformation Alimentaire&amp;Affaire : Société de Transformation Alimentaire(Conseils : SCPA LBTI &amp; PARTNERS, Avocats à la Cour)contreAc X BAb Ad CA DéveloppementArrêt N° 027/2021 du 25 février 2021La Cour Commune de Justice et d’Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxièmechambre, présidée par Monsieur Robert SAFARI ZIHALIRWA, assisté de Maître Koessy Alfred BADO, Greffier

### Traitement des fichiers JSON

La fonction ci-dessous montre comment traiter et charger les fichiers JSON dans un objet Dataset adapté à la construction de la base de données vectorielle.

In [ ]:
import os
import json
from datasets import Dataset

# Fonction pour traiter le contenu du fichier JSON
def process_json_data(json_data):
    # Adaptez ceci en fonction de la structure de votre fichier JSON
    # En supposant que le JSON contient des clés comme 'title', 'content' et 'details'
    title = json_data.get('title', 'Aucun Titre')  # Par défaut, 'Aucun Titre' si non trouvé
    content = json_data.get('content', 'Aucun Contenu')  # Par défaut, 'Aucun Contenu' si non trouvé
    details = json_data.get('details', 'Aucun Détail')  # Par défaut, 'Aucun Détail' si non trouvé

    return {'title': title, 'content': content, 'details': details}

# Répertoire contenant vos fichiers JSON
json_directory = JSONS_PATH

# Liste pour contenir les données traitées
jsons_data = []

# Itérer sur tous les fichiers JSON dans le répertoire
for json_file in os.listdir(json_directory):
    if json_file.endswith(".json"):
        json_path = os.path.join(json_directory, json_file)

        # Lire et traiter chaque fichier JSON
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

        # Traiter les données JSON et les ajouter à la liste
        processed_data = process_json_data(json_data)
        jsons_data.append(processed_data)

# Optionnel: Créer un objet Dataset avec les données traitées
dataset = Dataset.from_dict({'data': jsons_data})

# Afficher les premières lignes du Dataset pour vérifier
print(dataset[:5])

{'data': [{'content': 'ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICEET D’ARBITRAGE(CCJA)Deuxième chambreAudience publique du 25 février 2021Pourvois : n°219/2019/PC du 14/08/2019n°220/2019/PC du 14/08/2019Affaire : ONYX Développement(Conseil : Maître Agi LAWEL CHEKOU KORE, Avocat à la Cour)contreAc X BAb Ad CSociété de Transformation Alimentaire&amp;Affaire : Société de Transformation Alimentaire(Conseils : SCPA LBTI &amp; PARTNERS, Avocats à la Cour)contreAc X BAb Ad CA DéveloppementArrêt N° 027/2021 du 25 février 2021La Cour Commune de Justice et d’Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxièmechambre, présidée par Monsieur Robert SAFARI ZIHALIRWA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 25 février 2021, l’Arrêt dont la teneur suit, après délibération du collège de Juges composé de :Monsieur : Robert SAFARI ZIHALIRWA,   PrésidentMes

In [ ]:
# Maintenant, 'data' contient une liste de dictionnaires comme
# {'title': 'Titre 1', 'content': 'Contenu 1', 'details': 'Détails 1'}
print(jsons_data[:2])  # Afficher les deux premiers éléments traités

[{'title': "| OHADA, Cour commune de justice et d'arbitrage, 25 février 2021, 027/2021", 'content': 'ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICEET D’ARBITRAGE(CCJA)Deuxième chambreAudience publique du 25 février 2021Pourvois : n°219/2019/PC du 14/08/2019n°220/2019/PC du 14/08/2019Affaire : ONYX Développement(Conseil : Maître Agi LAWEL CHEKOU KORE, Avocat à la Cour)contreAc X BAb Ad CSociété de Transformation Alimentaire&amp;Affaire : Société de Transformation Alimentaire(Conseils : SCPA LBTI &amp; PARTNERS, Avocats à la Cour)contreAc X BAb Ad CA DéveloppementArrêt N° 027/2021 du 25 février 2021La Cour Commune de Justice et d’Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxièmechambre, présidée par Monsieur Robert SAFARI ZIHALIRWA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 25 février 2021, l’Arrêt dont la teneur suit, après délibération du c

In [ ]:
# Vérifier le contenu d'un élément de données donné
jsons_data[0]['title']

"| OHADA, Cour commune de justice et d'arbitrage, 25 février 2021, 027/2021"

In [ ]:
jsons_data[0]['content']

'ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICEET D’ARBITRAGE(CCJA)Deuxième chambreAudience publique du 25 février 2021Pourvois : n°219/2019/PC du 14/08/2019n°220/2019/PC du 14/08/2019Affaire : ONYX Développement(Conseil : Maître Agi LAWEL CHEKOU KORE, Avocat à la Cour)contreAc X BAb Ad CSociété de Transformation Alimentaire&amp;Affaire : Société de Transformation Alimentaire(Conseils : SCPA LBTI &amp; PARTNERS, Avocats à la Cour)contreAc X BAb Ad CA DéveloppementArrêt N° 027/2021 du 25 février 2021La Cour Commune de Justice et d’Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxièmechambre, présidée par Monsieur Robert SAFARI ZIHALIRWA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 25 février 2021, l’Arrêt dont la teneur suit, après délibération du collège de Juges composé de :Monsieur : Robert SAFARI ZIHALIRWA,   PrésidentMessieurs : Djimasna NDON

In [ ]:
jsons_data[0]['details']

"Références :\nOrigine de la décision\nPays :\nOHADA\nJuridiction :\nJuridiction :\nCour commune de justice et d'arbitrage\nDate de la décision :\n25/02/2021\nDate de l'import :\n01/06/2022\nNumérotation\nNuméro d'arrêt : 027/2021\nIdentifiant URN:LEX : urn:lex;ohada;cour.commune.justice.arbitrage;arret;2021-02-25;027.2021"

## Chargement des fichiers JSON dans un Dataset

Les données ont deux formats : pdfs et jsons. Le code ci-dessous montre comment charger les fichiers json pour create un object Dataset.

In [ ]:
from datasets import Dataset, DatasetDict

# Créer un Dataset à partir de la liste de dictionnaires
data_set = Dataset.from_dict({
    'title': [entry['title'] for entry in jsons_data],
    'content': [entry['content'] for entry in jsons_data],
    'details': [entry['details'] for entry in jsons_data],
})

# Diviser le dataset en train (ou autres splits si nécessaire)
data_set = DatasetDict({
    'train': data_set,
})

# Maintenant, le dataset_dict devrait contenir la structure souhaitée
print(data_set)

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'details'],
        num_rows: 1184
    })
})


In [ ]:
data_set['train'][0].keys()

dict_keys(['title', 'content', 'details'])

In [ ]:
print(data_set['train'][0]['title'])

| OHADA, Cour commune de justice et d'arbitrage, 25 février 2021, 027/2021


In [ ]:
print(data_set['train'][0]['content'])

ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICEET D’ARBITRAGE(CCJA)Deuxième chambreAudience publique du 25 février 2021Pourvois : n°219/2019/PC du 14/08/2019n°220/2019/PC du 14/08/2019Affaire : ONYX Développement(Conseil : Maître Agi LAWEL CHEKOU KORE, Avocat à la Cour)contreAc X BAb Ad CSociété de Transformation Alimentaire&amp;Affaire : Société de Transformation Alimentaire(Conseils : SCPA LBTI &amp; PARTNERS, Avocats à la Cour)contreAc X BAb Ad CA DéveloppementArrêt N° 027/2021 du 25 février 2021La Cour Commune de Justice et d’Arbitrage (CCJA) de l’Organisation pour l’Harmonisation en Afrique du Droit des Affaires (OHADA), Deuxièmechambre, présidée par Monsieur Robert SAFARI ZIHALIRWA, assisté de Maître Koessy Alfred BADO, Greffier, a rendu en son audience publique du 25 février 2021, l’Arrêt dont la teneur suit, après délibération du collège de Juges composé de :Monsieur : Robert SAFARI ZIHALIRWA,   PrésidentMessieurs : Djimasna NDONI

In [ ]:
print(data_set['train'][0]['details'])

Références :
Origine de la décision
Pays :
OHADA
Juridiction :
Juridiction :
Cour commune de justice et d'arbitrage
Date de la décision :
25/02/2021
Date de l'import :
01/06/2022
Numérotation
Numéro d'arrêt : 027/2021
Identifiant URN:LEX : urn:lex;ohada;cour.commune.justice.arbitrage;arret;2021-02-25;027.2021


In [ ]:
# texts = []  # Initialize texts as an empty list to store file contents

# for number in range(len(data_set['train'])):
#   content = data_set['train'][number]['content']
#   texts.append(content)

# print(f"The json folder consists of {len(texts)} texts.")

In [ ]:
#print(texts[0])

In [ ]:
#print(texts[-1])

## Extraire les textes des fichiers PDF

___Tâche___ : Lire et traiter les fichiers PDF du dossier pdf de manière similaire.


In [ ]:
# import os
# import pdfplumber

# # Fonction pour extraire le texte des fichiers PDF
# def extraire_texte_pdf(pdf_path):
#     """
#     Ouvre un fichier PDF et extrait le texte de toutes ses pages.
#     :param pdf_path: Chemin du fichier PDF à traiter
#     :return: Texte extrait de toutes les pages du PDF
#     """
#     texte_complet = ""
#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             texte = page.extract_text()
#             if texte:
#                 texte_complet += texte + "\n"
#     return texte_complet

# # Dossier contenant les fichiers PDF
# pdf_directory = PDFS_PATH

# # Liste pour stocker les textes extraits des fichiers PDF
# pdf_texts = []

# # Itérer sur tous les fichiers PDF dans le dossier
# for pdf_file in os.listdir(pdf_directory):
#     if pdf_file.endswith(".pdf"):
#         pdf_path = os.path.join(pdf_directory, pdf_file)

#         # Lire et extraire le texte de chaque fichier PDF
#         texte_pdf = extraire_texte_pdf(pdf_path)

#         # Ajouter le texte extrait à la liste
#         pdf_texts.append({
#             'filename': pdf_file,
#             'text': texte_pdf
#         })

# # Afficher les premiers textes extraits
# print(pdf_texts[:2])  # Afficher les deux premiers éléments extraits


In [ ]:
## extend the texts list with the pdf_texts list
# texts.extend(pdf_texts)
# print(f"The length of texts is now {len(texts)}.")

In [ ]:
# Charger tous les fichiers PDF dans le répertoire
loader = PyPDFDirectoryLoader(PDFS_PATH)

# Cela renverra une liste d'objets Document
pdf_documents = loader.load()

In [ ]:
# Afficher un document d'exemple
pdf_documents[0]

Document(metadata={'producer': 'Acrobat Distiller 10.1.1 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2017-11-30T14:41:00+01:00', 'author': 'Droit Afrique', 'moddate': '2017-11-30T14:42:00+01:00', 'title': 'OHADA - Acte uniforme relatif a la mediation (www.droit-afrique.com)', 'source': '/content/Data/data_pdfs/AU_mediation.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='www.droit‐afrique.com\t OHADA\n\t \t\n\t\nActe\tuniforme\trelatif\tà\tla\tmédiation\t 1\nOHADA\t\nActe\tuniforme\trelatif\tà\tla\tmédiation\t\nActe\tfait\tà\tConakry\tle\t23\tnovembre\t2017\t\n\t\n[NB\t‐\tActe\tuniforme\trelatif\tà\tla\tmédiation,\tsigné\tà\tConakry\tle\t23\tnovembre\t2017]\t\nChapitre\t1\t‐\tDéfinitions\tet\tchamp\td’application\t\nArt.1.‐\tDéfinitions\t\nAu\tsens\tdu\tprésent\tActe\tuniforme\t:\t\n\uf0b7\uf020 a)\tle\tterme\t«\tmédiation\t»\tdésigne\ttout\tprocessus,\tquelle\tque\tso it\tson\tappellation,\t\ndans\tlequel\tles\tparties\tdemandent\tà\tun\tt

In [ ]:
# Contenu d'un pdf
print(pdf_documents[0].page_content)

www.droit‐afrique.com	 OHADA
	 	
	
Acte	uniforme	relatif	à	la	médiation	 1
OHADA	
Acte	uniforme	relatif	à	la	médiation	
Acte	fait	à	Conakry	le	23	novembre	2017	
	
[NB	‐	Acte	uniforme	relatif	à	la	médiation,	signé	à	Conakry	le	23	novembre	2017]	
Chapitre	1	‐	Définitions	et	champ	d’application	
Art.1.‐	Définitions	
Au	sens	du	présent	Acte	uniforme	:	
 a)	le	terme	«	médiation	»	désigne	tout	processus,	quelle	que	so it	son	appellation,	
dans	lequel	les	parties	demandent	à	un	tiers	de	les	aider	à	par venir	à	un	règlement	
amiable	 d’un	 litige,	 d’un	 rapport	 conflictuel	 ou	 d’un	 désaccord	( c i ‐ a p r è s 	
le	«	différend	»)	découlant	d’un	rapport	juridique,	contractuel	ou	autre	ou	lié	à	un	tel	
rapport,	 impliquant	 des	 personnes	 physiques	 ou	 morales,	 y	 compris	 des	 entités	
publiques	ou	des	Etats	;	
 b)	 le	 terme	«	médiateur	»	désigne	 tout	 tiers	 sollicité	 pour	 mener	 une	 médiation	
quelle	que	soit	l’appellation	ou	la	profession	de	ce	tiers	dans	l’État	Partie	concern

In [ ]:
def extract_from_document(document):
    # Extraire le titre du document. Utiliser le titre dans les métadonnées ou la première ligne du contenu de la page
    title = document.metadata.get("title") or document.page_content.split("\n")[0]

    # Extraire les détails du document à partir des métadonnées
    details = {
        "author": document.metadata.get("author"),  # Auteur du document
        "creation_date": document.metadata.get("creationdate"),  # Date de création
        "producer": document.metadata.get("producer"),  # Producteur du document
        "creator": document.metadata.get("creator"),  # Créateur du document
        "modification_date": document.metadata.get("moddate"),  # Date de modification
        "source": document.metadata.get("source"),  # Source du document
    }

    # Nettoyer et extraire le contenu de la page
    content = document.page_content.strip()

    # Retourner un dictionnaire avec le titre, les détails et le contenu extraits
    return {
        "title": title,
        "details": details,
        "content": content
    }

In [ ]:
doc = extract_from_document(pdf_documents[0])
doc['title']

'OHADA - Acte uniforme relatif a la mediation (www.droit-afrique.com)'

In [ ]:
print(doc['content'])

www.droit‐afrique.com	 OHADA
	 	
	
Acte	uniforme	relatif	à	la	médiation	 1
OHADA	
Acte	uniforme	relatif	à	la	médiation	
Acte	fait	à	Conakry	le	23	novembre	2017	
	
[NB	‐	Acte	uniforme	relatif	à	la	médiation,	signé	à	Conakry	le	23	novembre	2017]	
Chapitre	1	‐	Définitions	et	champ	d’application	
Art.1.‐	Définitions	
Au	sens	du	présent	Acte	uniforme	:	
 a)	le	terme	«	médiation	»	désigne	tout	processus,	quelle	que	so it	son	appellation,	
dans	lequel	les	parties	demandent	à	un	tiers	de	les	aider	à	par venir	à	un	règlement	
amiable	 d’un	 litige,	 d’un	 rapport	 conflictuel	 ou	 d’un	 désaccord	( c i ‐ a p r è s 	
le	«	différend	»)	découlant	d’un	rapport	juridique,	contractuel	ou	autre	ou	lié	à	un	tel	
rapport,	 impliquant	 des	 personnes	 physiques	 ou	 morales,	 y	 compris	 des	 entités	
publiques	ou	des	Etats	;	
 b)	 le	 terme	«	médiateur	»	désigne	 tout	 tiers	 sollicité	 pour	 mener	 une	 médiation	
quelle	que	soit	l’appellation	ou	la	profession	de	ce	tiers	dans	l’État	Partie	concern

In [ ]:
print(doc['details'])

{'author': 'Droit Afrique', 'creation_date': '2017-11-30T14:41:00+01:00', 'producer': 'Acrobat Distiller 10.1.1 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'modification_date': '2017-11-30T14:42:00+01:00', 'source': '/content/Data/data_pdfs/AU_mediation.pdf'}


In [ ]:
print(doc['details']['source'])

/content/Data/data_pdfs/AU_mediation.pdf


## Construire une classe Document pour une base de données clé-vecteur

In [ ]:
class Document:
    def __init__(self, content, metadata=None):
        """
        Initialiser un document avec son contenu et ses métadonnées.

        :param content: Contenu du document (texte ou autre format)
        :param metadata: Métadonnées associées au document (facultatif, un dictionnaire vide si non fourni)
        """
        # Initialiser le contenu de la page du document
        self.page_content = content

        # Initialiser les métadonnées comme un dictionnaire vide si aucun n'est fourni
        self.metadata = metadata if metadata else {}

    def update_metadata(self, key, value):
        """
        Mettre à jour les métadonnées du document.

        :param key: La clé des métadonnées à ajouter ou modifier
        :param value: La valeur associée à la clé
        """
        self.metadata[key] = value

    def get_metadata(self):
        """
        Récupérer les métadonnées du document.

        :return: Dictionnaire des métadonnées
        """
        return self.metadata

    def get_content(self):
        """
        Récupérer le contenu du document.

        :return: Contenu du document
        """
        return self.page_content

    def __str__(self):
        """
        Retourner une représentation sous forme de chaîne de caractères du document.

        :return: Une chaîne représentant le titre et un extrait du contenu
        """
        return f"Document: {self.metadata.get('title', 'Sans titre')} - Extrait: {self.page_content[:100]}..."


In [ ]:
# Construire les documents a partir the contenus json.

json_documents = [
    Document(content=data['content'], metadata={"title": data['title'], "details": data['details']})
    for data in data_set['train']
]

In [ ]:
# Examinons un document
print(json_documents[0])

Document: | OHADA, Cour commune de justice et d'arbitrage, 25 février 2021, 027/2021 - Extrait: ORGANISATION POUR L’HARMONISATIONEN AFRIQUE DU DROIT DES AFFAIRES(OHADA)COUR COMMUNE DE JUSTICEET D’...


In [ ]:
# json_documents = [Document(content=text) for text in texts]

# Combiner les documents JSON et les documents PDF
documents = json_documents + pdf_documents

# Diviser les documents en petits morceaux
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)  # Modifiez le chunk_size et le chunk_overlap selon les besoins
all_splits = text_splitter.split_documents(documents)


In [ ]:
# spécifier le modèle d'encodage (en utilisant le transformateur de phrase de huggingface)

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)


<ipython-input-44-aead8a2ba6e4>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Exemple de texte pour obtenir l'embedding
text_to_embed = "Ceci est un exemple de texte pour obtenir un embedding."

# Obtenez l'embedding pour le texte
embedding = embeddings.embed_documents([text_to_embed])

# Affichez l'embedding
print(embedding)

[[-0.031500693410634995, -0.049962181597948074, 0.004357101861387491, 0.04359627515077591, -0.028997870162129402, 0.00850200280547142, 0.039947837591171265, 0.07866194099187851, 0.005423740483820438, -0.0007118612993508577, 0.039831679314374924, 0.02603030949831009, 0.012480353936553001, 0.0367625430226326, 0.029276421293616295, -0.05994526296854019, -0.017683643847703934, 0.018972592428326607, -0.011209827847778797, 0.012791404500603676, 0.022388296201825142, 0.013701763935387135, 0.01805119588971138, 0.015774231404066086, 0.026193635538220406, 0.03534393757581711, -0.0063852667808532715, -0.05070710927248001, 0.00973291601985693, -0.022381853312253952, -0.0014772039139643312, 0.01547527126967907, 0.05369507893919945, -0.05834585055708885, 1.6655454828651273e-06, 0.01438409835100174, -0.03820003196597099, 0.0064317574724555016, -0.006832440849393606, -0.04389461502432823, 0.013295079581439495, -0.024139169603586197, -0.0028641093522310257, -0.0194380059838295, -0.002373382216319442, 0

In [ ]:
folder_path2 = './Vector_db/'  # changez cela par le chemin dans votre disque où vous souhaitez stocker la base de données vectorielle

# intégrer les morceaux de documents
# les embeddings sont stockés dans une base de données (probablement sur disque) spécifiée par persist_directory
# assurez-vous que folder_path2 est correctement défini et pointe vers un répertoire valide où vous avez des permissions d'écriture
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory= folder_path2 + "chroma_db")
retrieverts = vectordb.as_retriever()

# Comment charger
# vectordbts = Chroma(persist_directory= folder_path2 + "chroma_db", embedding_function=embeddings)


# Créer un pipeline

In [ ]:
# Création d'un pipeline de génération de texte
text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# spécifier le modèle LLM
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

def create_conversation(query: str, chat_history: list, language: str = "French") -> tuple:
    try:
        # Objet pour stocker l'historique des conversations
        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )

        # Configuration de la chaîne de question-réponse
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retrieverts,
            memory=memory,
            get_chat_history=lambda h: h,
        )

        # Modifier la requête pour spécifier la langue de la réponse
        prompt = f"{query} Please answer in the {language} language."

        # Obtenir la réponse du LLM avec récupération et historique
        result = qa_chain.invoke({'question': prompt, 'chat_history': chat_history})

        # Ajouter la requête et la réponse à l'historique de la conversation
        chat_history.append((query, result['answer']))
        return '', chat_history

    except KeyError as ke:
        # Handle missing keys in the response or history
        error_message = f"Clé manquante dans la réponse ou l'historique: {ke}"
        chat_history.append((query, error_message))
        print(f"Error: {error_message}")
        return '', chat_history

    except ConnectionError as ce:
        # Handle any issues related to network or API calls
        error_message = f"Erreur de connexion: {ce}"
        chat_history.append((query, error_message))
        print(f"Network Error: {error_message}")
        return '', chat_history

    except Exception as e:
        # General exception handler for any other unexpected errors
        error_message = f"Erreur inattendue: {str(e)}"
        chat_history.append((query, error_message))
        print(f"Unexpected Error: {error_message}")
        return '', chat_history


Device set to use cuda:0
<ipython-input-47-0211b8cfbce5>:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
test_query = "Qu'est-ce que l'OHADA ?"
test_history = []
create_conversation(test_query, test_history)

<ipython-input-47-0211b8cfbce5>:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


('',
 [("Qu'est-ce que l'OHADA ?",
   "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nlesquelles, ayant estimé que l’affaire soulève des questions relatives à l’interprétation et à l’application du droit OHADA, s’étaient dessaisies par arrêt n° 05-1 03/CR du 22 avril 2005 au profit de la CCJA ;Sur le premier moyen pris en sa première brancheAttendu qu’il est fait grief à l’arrêt attaqué « d’avoir violé l’article 2 de la loi organique n° 62-11 du 16 mars 1962, par fausse motivation », en ce que l’arrêt du 18 août 2003 soutient que la réclamation de la propriété de la pharmacie par Madame ROUFAI est une demande nouvelle, pour avoir été introduite pour la première fois en Cour d’Appel, alors que, selon le moyen, dans ses conclusions du 15 janvier 2001, elle soutenait en page 6 qu’« au total, la cession de l’officine à Madame ROUFAI est intervenue dans les formes requises,

In [ ]:
test_history

[("Qu'est-ce que l'OHADA ?",
  "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nlesquelles, ayant estimé que l’affaire soulève des questions relatives à l’interprétation et à l’application du droit OHADA, s’étaient dessaisies par arrêt n° 05-1 03/CR du 22 avril 2005 au profit de la CCJA ;Sur le premier moyen pris en sa première brancheAttendu qu’il est fait grief à l’arrêt attaqué « d’avoir violé l’article 2 de la loi organique n° 62-11 du 16 mars 1962, par fausse motivation », en ce que l’arrêt du 18 août 2003 soutient que la réclamation de la propriété de la pharmacie par Madame ROUFAI est une demande nouvelle, pour avoir été introduite pour la première fois en Cour d’Appel, alors que, selon le moyen, dans ses conclusions du 15 janvier 2001, elle soutenait en page 6 qu’« au total, la cession de l’officine à Madame ROUFAI est intervenue dans les formes requises, ce qui

In [ ]:
from IPython.display import HTML

HTML("<span style='color:red; font-weight:bold'>L'extrait de code commenté dans la cellule suivante n'est pas censé être correct. Il vous donne simplement une idée de la manière de générer votre fichier de soumission.</span>")


In [ ]:
# import pandas as pd

# ## Constructing the submission csv
# submission_rows = []

# qid = "Q1"
# submission_rows.append({"ID": f"{qid}_Answer", "Target": answer})
# submission_rows.append({"ID": f"{qid}_Reference_Document", "Target": reference_document})
# submission_rows.append({"ID": f"{qid}_Article_Number", "Target": article_number})

# # turn submission_rows in a pd
# submission_df = pd.DataFrame(submission_rows)
# submission_df.to_csv("submission.csv", index=False)

# submission_df

# Prompt-Engineering

In [ ]:
test_query = "Qu'est-ce que l'OHADA ?"
test_prompt = f"""<s>[INST] {test_query}  [/INST] """
test_history = []
create_conversation(test_prompt, test_history)

('',
 [("<s>[INST] Qu'est-ce que l'OHADA ?  [/INST] ",
   'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nl’\'OHADA ;Attendu, selon les énonciations de l’arrêt attaqué, que suite à la saisie attribution de créances pratiquée le 30 octobre 2020 sur ses avoirs bancaires à la demande de la SCI Notre Dame de la Grâce, l’Ab A de Management dite AFRAM a fait assigner cette dernière devant le juge de l’exécution en contestation et mainlevée de la saisie ;Que le 27 novembre 2020, le juge de l’exécution a rendu son ordonnance par laquelle il a rejeté les prétentions de AFRAM et prononcé la continuation des poursuites ; que sur appel de celle-ci, la Cour d’appel judiciaire de Libreville a rendu l’arrêt objet du présent recours ;Sur le moyen unique de cassation tiré de la violation de la loiAttendu qu’il est reproché à l’arrêt attaqué d’avoir « confirmé pour le reste la décisi

In [ ]:
query = "Quel est le document de reference dans le quel se trouve la reponse a cette question ? Ne retourne que la reponse a ma question est rien d'autre"
prompt = f"""<s>[INST] {query}  [/INST] """
create_conversation(prompt, test_history)

('',
 [("<s>[INST] Qu'est-ce que l'OHADA ?  [/INST] ",
   'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nl’\'OHADA ;Attendu, selon les énonciations de l’arrêt attaqué, que suite à la saisie attribution de créances pratiquée le 30 octobre 2020 sur ses avoirs bancaires à la demande de la SCI Notre Dame de la Grâce, l’Ab A de Management dite AFRAM a fait assigner cette dernière devant le juge de l’exécution en contestation et mainlevée de la saisie ;Que le 27 novembre 2020, le juge de l’exécution a rendu son ordonnance par laquelle il a rejeté les prétentions de AFRAM et prononcé la continuation des poursuites ; que sur appel de celle-ci, la Cour d’appel judiciaire de Libreville a rendu l’arrêt objet du présent recours ;Sur le moyen unique de cassation tiré de la violation de la loiAttendu qu’il est reproché à l’arrêt attaqué d’avoir « confirmé pour le reste la décisi

# Gradio UI

In [ ]:
# Construire l'interface Gradio
with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='Chat with your Ohada dataset ;)')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    language = gr.State("French")
    msg.submit(create_conversation, [msg, chatbot, language], [msg, chatbot])

demo.launch()

<ipython-input-55-7fd69152d3c8>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label='Chat with your Ohada dataset ;)')


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b56b76b2e866e3fa08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Le fichier `Test.csv` contient une liste de questions auxquelles vous devez répondre à l’aide de votre LLM. Pour chaque question, vous devez indiquer quel est le Document de Référence contenant la réponse. Il faut également mentionner l’article dans lequel la réponse est donnée, si possible.

Le **Document de Référence** correspond au titre du document contenant la réponse.

Le **Numéro_Article** correspond à l'article ou aux articles dans le(s)quel(s) la réponse est mentionnée.

Vous devez soumettre vos réponses dans un fichier CSV ayant une structure similaire à celle du fichier `SampleSubmission.csv`.

In [ ]:
import pandas as pd

ss = pd.read_csv("SampleSubmission.csv")
ss.head()

,ID,Target
0,Q1_Answer,NaN
1,Q1_Reference_Document,NaN
2,Q1_Article_Number,NaN
3,Q2_Answer,NaN
4,Q2_Reference_Document,NaN


### Bonne chance pour la compétition! 🚀


### Soyez Creatifs!